In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
path = "Team Banksia_ Finding Australia’s Space Niche (Responses) - Form Responses.csv"
DataFrame = pd.read_csv(path)

# Next Steps
1. Convert MCs to Numeric
2. Scatter Plots
3. Pearson Correlation Coefficient


In [5]:
# Show the DataFrame
DataFrame.head()

,Timestamp,Where are you in your career,What field are you in?,How familiar are you with Australia's space sector?,Does the Australian space program have relevance to you?,"How many ""Space Companies"" do you think there are in Australia",Do you think the Australian Space Industry is:,Do you believe space is important for Australia? Why/why not?,"In what ways do you expect the ""space sector"" to help your life in Australia?",What questions do YOU have about the Australian Space Industry?
0,1/8/2025 19:59:12,Postgrad / Masters Student,STEM,Somewhat,Yes,10-99,Stable,"Yes, there is a lot of potential for data link...",A good pool of jobs available for me after my ...,"I see a lot of communication based satellites,..."
1,1/9/2025 11:47:21,Just interested in space,Non-STEM,Not at all,Not sure,<10,Growing,Defence,NaN,NaN
2,1/9/2025 11:48:57,Working in Space Industry,STEM,Very,Yes,100-900,Growing,"Yes extremely. Sovereign capability, environme...","Space enables our everyday lives, like communi...","Who are the companies, industry leaders and ac..."
3,1/9/2025 12:17:42,Just interested in space,STEM,Somewhat,No,10-99,Growing,Yes - spaced based technologies will become mo...,Not sure.,"What do you bring to the industry that US, EU,..."
4,1/9/2025 13:18:49,Undergrad Student (Bachelors/Honours),BOTH,Not at all,Not sure,<10,Stable,yes,NaN,NaN


In [13]:
# Build a numeric DF
NumericColumns = ["User", "Career", "Field", "Familiarity", "Relevance", "Space Company Estimate", "Growth"]
NumericDataFrame = pd.DataFrame(columns=NumericColumns)

NumericDataFrame.head()

# Fill Dictionaries With Definitions
CareerDict = {
    "Undergrad Student (Bachelors/Honours)": 0 ,
    "Postgrad / Masters Student": 1 , 
    "Working in Space Industry": 2 ,
    "Own a space port and research organisation": 2, #Lumping this in with working in the space industry
    "Just interested in space": 3 ,
    "Working in Academia": 4 ,
}

FieldDict = {
    "STEM": 0,
    "Non-STEM": 1,
    "BOTH": 2
}

FamiliarityDict = {
    "Not at all": 0,
    "Not Really": 1,
    "Somewhat": 2,
    "Very": 3}

RelevanceDict = {
    "Yes": 1,
    "No": 0,
    "Not Sure": 0, # Lump not sure in with no
}

SpaceCompanyEstimateDict = {
 "<10": 1,
 "10-99": 10,
 "100-900": 100,
 ">1000": 1000
 }

GrowthDict = {
    "Non-existent": 0,
    "Shrinking": 1,
    "Stable": 2,
    "Growing": 3
}

for row in DataFrame.iterrows():
    index, data = row
    
    Career = CareerDict[data["Where are you in your career"]]
    print(index, Career)

SyntaxError: '{' was never closed (2598662326.py, line 35)